Mini Transformer - BUMBLEBEE - CEL: zbudować mały model jezykowy na self-attension z jedną warstwą transformera, która potrafi uczyć się prostych zależnosci w sekwencjach (przwidywanie następnego tokenu)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

KROK2: definiujemy poejdynczy blok transformera

In [8]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_dim):
        super(TransformerBlock, self).__init__()
        self.key = nn.Linear(embed_dim, embed_dim,bias=False)
        self.query = nn.Linear(embed_dim, embed_dim,bias=False)
        self.value = nn.Linear(embed_dim, embed_dim,bias=False)
        self.proj = nn.Linear(embed_dim, embed_dim)
        self.ln = nn.LayerNorm(embed_dim)

    #KROK3: mechanizm self-attention

    def forward(self, x):
        B, T, C = x.shape #B=batch, T=sequence length, C=embedding size
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)

        scores = q @ k.transpose(-2, -1) / (C**0.5) #B,T,T -> oblicza podobieństwo między tokenami!
        mask = torch.tril(torch.ones(T,T)).to(x.device) #nie patrzymy na przyszłe tokeny - (ważne w modelach generatywnych)
        scores = scores.masked_fill(mask == 0,float('-inf')) #maskowanie przyszłości

        att = F.softmax(scores, dim=-1) #rozkład uwagi
        out=att@v #mieszanie wartości zgodnie z uwagą , v to "treść" tokenu który bierzemy
        return self.ln(self.proj(out) + x) #resztkowe połączenie i normalizacja



KROK 4: budujemy minimodel - Mini Transformer

In [9]:
class MiniTransformer(nn.Module):
    def __init__(self, vocab_size, embed_dim,block_size):
        super(MiniTransformer, self).__init__()
        self.token_emb= nn.Embedding(vocab_size, embed_dim) #zmienia ID tokenu na wektor(embedding)
        self.pos_emb = nn.Embedding(block_size, embed_dim) #dodaje informację o kolejności tokenów
        self.transformer = TransformerBlock(embed_dim) #jeden blok attention + LayerNorm
        self.lm_head = nn.Linear(embed_dim, vocab_size) #przekształca wektor z powrotem na prawdopodobieństwo tokenów

    #KROK5: forward - przetwarzanie sekwencji

    def forward(self, idx):
      B, T = idx.shape
      token_embeddings = self.token_emb(idx) #(B,T,C)
      position_embeddings = self.pos_emb(torch.arange(T).to(idx.device)) #(T,C)
      x = token_embeddings + position_embeddings #(B,T,C) - dodwanie pozycji
      x = self.transformer(x) #(B,T,C)
      logits = self.lm_head(x) #(B,T,vocab_size) - -predykcja kolejnych tokenów
      return logits

    #model bierze sekwencję tokenów, przelicza ich znaczenie (token + pozycja), przetwarza przez mechanizm attention
    #i na końcu mówi: jaki powinien być następny token

In [10]:
#użycie modelu
model = MiniTransformer(vocab_size=20, embed_dim=32, block_size=8)
example_input = torch.randint(0, 20, (1, 8))
output = model(example_input)
print(output.shape)
print(f"Input: {example_input}")
print(f"Output: {output}")

torch.Size([1, 8, 20])
Input: tensor([[ 2, 18, 11, 18, 14, 11, 17, 16]])
Output: tensor([[[ 3.7593e-01,  5.2333e-01,  8.2774e-01,  3.3304e-01,  5.6342e-01,
          -1.4758e-01, -5.1432e-02, -1.2750e-01, -5.8661e-01,  7.7366e-01,
           2.9653e-01,  7.3321e-01,  7.9648e-01, -1.5888e-01, -6.4371e-01,
          -1.0302e-02, -4.6319e-01, -7.0561e-01, -2.6548e-01,  1.0039e-01],
         [-6.8498e-01,  6.1105e-01,  1.2246e+00,  1.0107e-01,  2.2005e-01,
           3.2313e-02,  6.6484e-01,  2.2130e-04, -1.7977e-01,  2.3000e-01,
           6.7188e-01, -1.2369e-01, -2.2323e-01,  8.4430e-01, -1.0059e+00,
           4.6596e-01, -3.0223e-01,  1.0519e-01, -3.5162e-01, -1.3727e+00],
         [ 6.3291e-02,  5.9362e-01,  6.9861e-01, -2.4390e-02, -7.2545e-02,
          -2.2119e-01, -2.8862e-02,  3.6147e-02,  3.2116e-02, -1.0930e+00,
           3.0994e-01,  6.5896e-01,  9.1561e-01, -2.5862e-01, -1.5541e-01,
           5.6020e-01, -6.4970e-01, -4.7901e-01, -5.4097e-01,  5.9125e-01],
         [-3.083